In [5]:
import pandas as pd
import xgboost as xgb
import numpy as np
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier
from sklearn.metrics import (
    f1_score, 
    precision_score, 
    recall_score, 
    accuracy_score, 
    classification_report
)
from sklearn.model_selection import StratifiedKFold
import numpy as np
import pickle

In [6]:
data = pd.read_pickle("/Users/mac/Desktop/Code_Smell_Detection/dataset/feature_envy/cubert_embeddings/processed_embeddings/cubert_embed_line_by_line_sum.pkl")
data.head()

,emb_1,emb_2,emb_3,emb_4,emb_5,emb_6,emb_7,emb_8,emb_9,emb_10,...,emb_762,emb_763,emb_764,emb_765,emb_766,emb_767,emb_768,label,sample_id,severity
0,-42.401798,-26.870686,-46.018105,8.521990,15.175569,-18.864988,-5.143667,-103.999161,52.696270,34.906448,...,14.058058,-35.712307,-3.190466,49.794823,-42.799503,48.233025,-43.109947,1,12150,critical
1,4.771414,-9.410942,-2.367512,0.598740,-5.552438,-2.216153,2.808177,-3.373450,4.429100,3.801935,...,0.464715,2.624750,-0.605876,2.401019,-3.718243,6.333813,-4.118282,1,10168,critical
2,11.647195,-80.076416,-124.189705,58.185944,48.472561,24.540005,36.780075,-63.776188,82.424889,20.396940,...,31.705832,2.266652,38.338787,61.845779,-0.191301,130.945633,-39.840389,1,12783,critical
3,-6.452082,-117.957657,-129.250961,41.561977,-21.422041,-83.653915,104.651474,-106.414589,20.539492,-15.075366,...,-34.709187,-47.284946,-4.274190,50.157265,-34.449261,68.198784,-8.792969,1,12754,critical
4,8.611705,-30.427801,-61.426537,33.076874,20.033140,-68.134056,14.036429,-13.502237,31.836353,34.797176,...,21.287672,24.752357,2.240633,23.209963,-0.617386,50.764732,-25.916571,1,4277,critical


In [7]:
# Separate features and labels
X = data[[col for col in data.columns if col.startswith("emb_")]].values  # Select embedding columns
y = data['label'].values 

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

# Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Define classifiers
random_forest = RandomForestClassifier(n_estimators=100, random_state=42)
bagging = BaggingClassifier(estimator=RandomForestClassifier(), n_estimators=100, random_state=42)  
xgboost = xgb.XGBClassifier(n_estimators=100, random_state=42)

# Add the classifiers to a dictionary
classifiers = {
    "Random Forest": random_forest,
    "Bagging": bagging,
    "XGBoost": xgboost
}

In [8]:
# Initialize StratifiedKFold
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Dictionary to store cross-validation results
results = {}

# Perform cross-validation for each classifier
for clf_name, clf in classifiers.items():
    metrics_summary = {
        "f1_micro": [],
        "f1_macro": [],
        "accuracy": [],
        "precision_macro": [],
        "recall_macro": []
    }
    
    for train_idx, val_idx in skf.split(X_train, y_train):
        # Split the training data into training and validation folds
        X_train_fold, X_val_fold = X_train[train_idx], X_train[val_idx]
        y_train_fold, y_val_fold = y_train[train_idx], y_train[val_idx]
        
        # Train the classifier
        clf.fit(X_train_fold, y_train_fold)
        
        # Make predictions on the validation set
        y_pred = clf.predict(X_val_fold)
        
        # Calculate metrics
        metrics_summary["f1_micro"].append(f1_score(y_val_fold, y_pred, average='micro'))
        metrics_summary["f1_macro"].append(f1_score(y_val_fold, y_pred, average='macro'))
        metrics_summary["accuracy"].append(accuracy_score(y_val_fold, y_pred))
        metrics_summary["precision_macro"].append(precision_score(y_val_fold, y_pred, average='macro'))
        metrics_summary["recall_macro"].append(recall_score(y_val_fold, y_pred, average='macro'))
    
    # Store mean metrics for the classifier
    results[clf_name] = {metric: np.mean(scores) for metric, scores in metrics_summary.items()}

# Print cross-validation results
print("\nCross-Validation Results:")
for clf_name, metrics in results.items():
    print(f"\n{clf_name} Metrics:")
    for metric, value in metrics.items():
        print(f"{metric.capitalize()}: {value:.4f}")


Cross-Validation Results:

Random Forest Metrics:
F1_micro: 0.8314
F1_macro: 0.5808
Accuracy: 0.8314
Precision_macro: 0.6017
Recall_macro: 0.5720

Bagging Metrics:
F1_micro: 0.8432
F1_macro: 0.5555
Accuracy: 0.8432
Precision_macro: 0.6029
Recall_macro: 0.5480

XGBoost Metrics:
F1_micro: 0.8314
F1_macro: 0.5693
Accuracy: 0.8314
Precision_macro: 0.5920
Recall_macro: 0.5618


In [9]:
# Final evaluation on the test set
final_results = {}

for clf_name, clf in classifiers.items():
    # Train the classifier on the entire training set
    clf.fit(X_train, y_train)
    
    # Make predictions on the test set
    y_pred_test = clf.predict(X_test)
    
    # Calculate test metrics
    f1_micro_test = f1_score(y_test, y_pred_test, average='micro')
    f1_macro_test = f1_score(y_test, y_pred_test, average='macro')
    precision_test = precision_score(y_test, y_pred_test, average='macro')
    recall_test = recall_score(y_test, y_pred_test, average='macro')
    accuracy_test = accuracy_score(y_test, y_pred_test)
    report_test = classification_report(y_test, y_pred_test)
    
    # Save final metrics for the classifier
    final_results[clf_name] = {
        "f1_micro": f1_micro_test,
        "f1_macro": f1_macro_test,
        "precision_macro": precision_test,
        "recall_macro": recall_test,
        "accuracy": accuracy_test,
        "classification_report": report_test
    }

# Print final test results
print("\nFinal Test Results:")
for clf_name, metrics in final_results.items():
    print(f"\n{clf_name} Metrics:")
    for metric, value in metrics.items():
        if metric != "classification_report":
            print(f"{metric.capitalize()}: {value:.4f}")
    print(f"Classification Report:\n{metrics['classification_report']}")



Final Test Results:

Random Forest Metrics:
F1_micro: 0.8196
F1_macro: 0.5621
Precision_macro: 0.5745
Recall_macro: 0.5562
Accuracy: 0.8196
Classification Report:
              precision    recall  f1-score   support

           0       0.88      0.92      0.90       786
           1       0.27      0.20      0.23       123

    accuracy                           0.82       909
   macro avg       0.57      0.56      0.56       909
weighted avg       0.80      0.82      0.81       909


Bagging Metrics:
F1_micro: 0.8438
F1_macro: 0.5622
Precision_macro: 0.6056
Recall_macro: 0.5531
Accuracy: 0.8438
Classification Report:
              precision    recall  f1-score   support

           0       0.88      0.95      0.91       786
           1       0.33      0.15      0.21       123

    accuracy                           0.84       909
   macro avg       0.61      0.55      0.56       909
weighted avg       0.80      0.84      0.82       909


XGBoost Metrics:
F1_micro: 0.8207
F1_macro: 

In [11]:
# Save results to a pickle file
output_path = "/Users/mac/Desktop/Code_Smell_Detection/feature_envy/results/cubert/result_cubert_sum.pkl"
with open(output_path, "wb") as f:
    pickle.dump({"cross_val_results": results, "final_results": final_results}, f)
print(f"\nResults saved to {output_path}")



Results saved to /Users/mac/Desktop/Code_Smell_Detection/feature_envy/results/cubert/result_cubert_sum.pkl
